In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
# 영어 단어의 어근만 추출
stm = PorterStemmer()
# 영어 단어의 불용어 집합
stopwords = set(stopwords.words('english'))
# 특수문자를 제거하기 위한 정규식
# 첫글자는 알파벳으로 시작하고 그 뒤에 영문자 대소문자, 숫자, -,_, .만 허용
# * 0회 이상 매칭, + 1회 이상 매칭
pattern = re.compile('[a-zA-Z][-_a-zA-Z0-9.]*')
# 문장을 단어 단위로 분리하고 불용어 및 특수문자 제거 후 어근만 추출하여 list로 반환
def tokenize(sentence):
    def stem(w):
        try: return stm.stem(w)
        except: return w
    #소문자로 바꾼 후 단어 구분, 불용어 제거, 패턴에 맞는 단어만 선택
    return [stem(w) for w in word_tokenize(sentence.lower())
        if w not in stopwords and pattern.match(w)]

In [ ]:
import tomotopy as tp
# LDAModel 생성
# LDA(Latent Dirichlet allocation, 잠재 디리클레 할당)
# 주어진 문서에 대하여 각 문서에 어떤 주제들이 존재하는지를 서술하는 확률적 토픽 모델 기법
# 토픽의 개수(k) 20개, 5회 미만 등장한 단어들은 제거
model = tp.LDAModel(k=20, min_cf=5)
# 파일에서 한 줄씩 읽어와서 model에 추가
for i, line in enumerate(open('c:/data/text/trumph.txt', encoding='ms949')):
    model.add_doc(tokenize(line)) # 공백 기준으로 단어를 나누어 model에 추가
    # train(0) : 0회 학습, model의 num_words, num_vocabs 값을 확인하기 위해
    # 실제로 학습은 하지 않고 학습 준비만 하는 상태
model.train(0)
print('Total docs:', len(model.docs))
print('Total words:', model.num_words)
print(model.vocabs) #단어 리스트

In [ ]:
#200회 학습
model.train(200)
# 학습된 토픽들을 출력
for i in range(model.k):
    # 0~19번 토픽별 상위 단어 10개 추출
    res = model.get_topic_words(i, top_n=10)
    print(f'Topic #{i}', end='\t')
    print(', '.join(w for w, p in res))

In [4]:
import nltk
emma_raw=nltk.corpus.gutenberg.raw('austen-emma.txt')
tokenize(emma_raw)[:100]

['emma',
 'jane',
 'austen',
 'volum',
 'chapter',
 'emma',
 'woodhous',
 'handsom',
 'clever',
 'rich',
 'comfort',
 'home',
 'happi',
 'disposit',
 'seem',
 'unit',
 'best',
 'bless',
 'exist',
 'live',
 'nearli',
 'twenty-on',
 'year',
 'world',
 'littl',
 'distress',
 'vex',
 'youngest',
 'two',
 'daughter',
 'affection',
 'indulg',
 'father',
 'consequ',
 'sister',
 'marriag',
 'mistress',
 'hous',
 'earli',
 'period',
 'mother',
 'die',
 'long',
 'ago',
 'indistinct',
 'remembr',
 'caress',
 'place',
 'suppli',
 'excel',
 'woman',
 'gover',
 'fallen',
 'littl',
 'short',
 'mother',
 'affect',
 'sixteen',
 'year',
 'miss',
 'taylor',
 'mr.',
 'woodhous',
 'famili',
 'less',
 'gover',
 'friend',
 'fond',
 'daughter',
 'particularli',
 'emma',
 'intimaci',
 'sister',
 'even',
 'miss',
 'taylor',
 'ceas',
 'hold',
 'nomin',
 'offic',
 'gover',
 'mild',
 'temper',
 'hardli',
 'allow',
 'impos',
 'restraint',
 'shadow',
 'author',
 'long',
 'pass',
 'away',
 'live',
 'togeth',
 'friend

In [6]:
model=tp.LDAModel(k=5, min_cf=5) #토픽모델링 함수
model.add_doc(tokenize(emma_raw)) #단어별로 나누고 모형에 추가
model.train(0) # 0 학습횟수, 아직 학습이 시작된 상태가 아님
print('Total docs:',len(model.docs))
print('Total words:',model.num_words)

Total docs: 1
Total words: 67220


In [8]:
list(model.vocabs)[0:5]

['mr.', 'emma', 'could', 'would', 'mrs.']

In [9]:
model.train(100) #100회 학습
for i in range(model.k):
    res=model.get_topic_words(i,top_n=2) #토픽의 상위단어 2개
    print(res)
    print(f'Topic #{i}',end='\t')
    print(', '.join(w for w,p in res))

[('everi', 0.030207434669137), ('elton', 0.02763812616467476)]
Topic #0	everi, elton
[('one', 0.03213750571012497), ('harriet', 0.022232817485928535)]
Topic #1	one, harriet
[('must', 0.0336933359503746), ('say', 0.032937146723270416)]
Topic #2	must, say
[('could', 0.030469197779893875), ('never', 0.02862989902496338)]
Topic #3	could, never
[('would', 0.03684122487902641), ('mr.', 0.03221023827791214)]
Topic #4	would, mr.


In [ ]:
# 알고리즘에 의해 관련성 있는 단어들끼리 묶은 토픽들
# 토픽 세부 내용을 보고 어떤 주제인지 확인하는 작업이 필요함
# 첫번째 토픽의 중심단어 could,know
# could는 6.1%, know는 2.8% 등장
# [('could', 0.0612008310854435), ('know',0.028761839494109154)]
# Topic #0 could, know
# 두번째 토픽의 중심단어는 mrs.(4.6%) miss(4.2%)
# [('mrs.', 0.04658449441194534), ('miss', 0.04260953143239021)]
# Topic #1 mrs., miss